# Part 1

This part is dedicated for preprocessing collections library documentation\
The original can be found here https://docs.python.org/3/library/collections.html

In [48]:
from pprint import pprint
from tqdm.auto import tqdm
from haystack.nodes import QuestionGenerator, BM25Retriever, FARMReader
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.pipelines import (
    QuestionGenerationPipeline,
    RetrieverQuestionGenerationPipeline,
    QuestionAnswerGenerationPipeline,
)
from haystack.utils import launch_es, print_questions

In [2]:
import re

RUBBISH = ['@']
def parse_dict_to_list(d, level=None, curr_level=0):
    result = []
    if isinstance(d, dict):
        for k, v in d.items():
            if isinstance(v, str) and k[0] not in RUBBISH:
                result.append(re.sub('^\s+|\s+$|\s+(?=\s)', ' ', v.strip()))
            elif isinstance(v, list):
                for item in v:
                    if isinstance(item, str) and k[0] not in RUBBISH:
                        result.append(re.sub('^\s+|\s+$|\s+(?=\s)', ' ', item.strip()))
                    else:
                        sub_result = parse_dict_to_list(item, level, curr_level + 1)
                        if curr_level < level:
                            result.extend(sub_result)
                        else:
                            result.append(' '.join(sub_result))
            else:
                sub_result = parse_dict_to_list(v, level, curr_level + 1)
                result.extend(sub_result)
        if curr_level==level:
            result=['. '.join(result)]
    return result

In [3]:
import json

collections_json = json.loads(open('../data/collections.json', 'r').read())

In [4]:
documents = parse_dict_to_list(collections_json, level=5)
print(len(documents))
# documents

147


In [78]:
launch_es()

DEBUG:haystack.utils.doc_store:Starting Elasticsearch ...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [93]:
docs = [{"content": x} for x in documents if len(x) > 100]

# Initialize document store and write in the documents
document_store = ElasticsearchDocumentStore()
document_store.delete_all_documents()
document_store.write_documents(docs)

/Users/khumoyunaminaddinov/PycharmProjects/Ai_inno/NLPaML_chatbot/venv/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
                1. delete_all_documents() method is deprecated, please use delete_documents method
                For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/1045
                


In [94]:
len(document_store.get_all_documents())

86

In [92]:
len(docs)

86

In [95]:
# Initialize Question Generator
question_generator = QuestionGenerator()

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
/Users/khumoyunaminaddinov/PycharmProjects/Ai_inno/NLPaML_chatbot/venv/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Using sep_token, but it is not set yet.


In [96]:
questions = []
answers = []

In [97]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.DEBUG)
logging.getLogger("haystack").setLevel(logging.DEBUG)


In [98]:
reader = FARMReader("deepset/roberta-base-squad2")
qag_pipeline = QuestionAnswerGenerationPipeline(question_generator, reader)
for idx, document in tqdm(enumerate(document_store)):

    print(f"\n * Generating questions and answers for document {idx}: {document.content[:100]}...\n")
    result = qag_pipeline.run(documents=[document])
    print_questions(result)

    for i, ans in enumerate(result['answers']):
        if ans:
            for a in ans:
                answers.append(a)
                questions.append(result['queries'][i])

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
DEBUG:haystack.modeling.model.prediction_head:Prediction head initialized with size [768, 2]
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


0it [00:00, ?it/s]

DEBUG:haystack.pipelines.base:Running node 'Query` with input: {'root_node': 'Query', 'params': None, 'documents': [<Document: {'content': 'This module implements specialized container datatypes providing alternatives to  Python’s general purpose built-in containers, dict, list,  set, and tuple.', 'content_type': 'text', 'score': 0.5312093733737563, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '695413c8a4659ba29cec6ad6dcf2dc7e'}>], 'node_id': 'Query'}
DEBUG:haystack.pipelines.base:Running node 'QuestionGenerator` with input: {'root_node': 'Query', 'params': {}, 'documents': [<Document: {'content': 'This module implements specialized container datatypes providing alternatives to  Python’s general purpose built-in containers, dict, list,  set, and tuple.', 'content_type': 'text', 'score': 0.5312093733737563, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '695413c8a4659ba29cec6ad6dcf2dc7e'}>], 'node_id': 'QuestionGenerator'}



 * Generating questions and answers for document 0: This module implements specialized container datatypes providing alternatives to  Python’s general p...



DEBUG:haystack.pipelines.base:Running node 'Reader` with input: {'documents': [[<Document: {'content': 'This module implements specialized container datatypes providing alternatives to  Python’s general purpose built-in containers, dict, list,  set, and tuple.', 'content_type': 'text', 'score': 0.5312093733737563, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '695413c8a4659ba29cec6ad6dcf2dc7e'}>], [<Document: {'content': 'This module implements specialized container datatypes providing alternatives to  Python’s general purpose built-in containers, dict, list,  set, and tuple.', 'content_type': 'text', 'score': 0.5312093733737563, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '695413c8a4659ba29cec6ad6dcf2dc7e'}>]], 'queries': ['What module implements specialized container datatypes?', "What are Python's general purpose built in containers?"], 'root_node': 'Query', 'params': {}, 'node_id': 'Reader'}
DEBUG:haystack.modeling.data_handler.dataset:Coul

...

DEBUG:haystack.pipelines.base:Running node 'Query` with input: {'root_node': 'Query', 'params': None, 'documents': [<Document: {'content': 'seq seq Class that simulates a string object.  The instance’s  content is kept in a regular string object, which is accessible via the  data attribute of UserString instances.  The instance’s  contents are initially set to a copy of .  The  argument can  be any object which can be converted into a string using the built-in  str() function.. In addition to supporting the methods and operations of strings,  UserString instances provide the following attribute:. data. A real str object used to store the contents of the  UserString class.. Changed in version 3.5: New methods __getnewargs__, __rmod__, casefold,  format_map, isprintable, and maketrans.', 'content_type': 'text', 'score': 0.5312093733737563, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '525c01b28b3e750002f20f252a536e02'}>], 'node_id': 'Query'}
DEBUG:haystack.pipelines.


Generated pairs:
 - Q: What class acts as a wrapper around string objects?
      A: UserString
 - Q: What has partially supplanted the need for this class?
      A: the ability to  subclass directly from str

 * Generating questions and answers for document 85: seq seq Class that simulates a string object.  The instance’s  content is kept in a regular string o...



DEBUG:haystack.pipelines.base:Running node 'Reader` with input: {'documents': [[<Document: {'content': 'seq seq Class that simulates a string object.  The instance’s  content is kept in a regular string object, which is accessible via the  data attribute of UserString instances.  The instance’s  contents are initially set to a copy of .  The  argument can  be any object which can be converted into a string using the built-in  str() function.. In addition to supporting the methods and operations of strings,  UserString instances provide the following attribute:. data. A real str object used to store the contents of the  UserString class.. Changed in version 3.5: New methods __getnewargs__, __rmod__, casefold,  format_map, isprintable, and maketrans.', 'content_type': 'text', 'score': 0.5312093733737563, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '525c01b28b3e750002f20f252a536e02'}>], [<Document: {'content': 'seq seq Class that simulates a string object.  The insta

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]


Generated pairs:
 - Q: What is a seq seq Class that simulates a string object?
      A: UserString instances
 - Q: How is the instance's content kept?
      A: in a regular string object
 - Q: What is accessible via the data attribute of UserString instances?
      A: The instance’s  content is kept in a regular string object
 - Q: What can be converted into a string using the built-in str() function?
      A: any object
 - Q: What do UserString instances provide?
      A: the following attribute:. data
 - Q: What is a real str object used to store the contents of the UserString class?
      A: data
 - Q: What new methods were added in version 3.5?
      A: __getnewargs__, __rmod__, casefold,  format_map, isprintable, and maketrans


In [101]:
sum([not isinstance(x, list) for x in answers[:121]]) == len(answers[:121])

True

In [102]:
len(questions), len(answers)

(689, 689)

In [103]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()

    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = " ".join(text.split())
    return text

In [113]:
import pandas as pd

dataset = pd.DataFrame(columns=['question', 'answer', 'context'])
for q, a in zip(questions, answers):
    tdsts = pd.DataFrame([[q, a.answer, a.context]], columns=['question', 'answer', 'context'], index=[len(dataset)])
    dataset = pd.concat([dataset, tdsts])
len(dataset)

689

In [115]:
dataset.head(4)

,question,answer,context
0,What module implements specialized container datatypes?,Python,odule implements specialized container datatypes providing alternatives to ...
1,What are Python's general purpose built in containers?,"dict, list, set, and tuple",odule implements specialized container datatypes providing alternatives to ...
2,What is the factory function for creating tuple subclasses with named fields?,namedtuple(),namedtuple() factory function for creating tuple subclasses with named field...
3,What is a deque list-like container with fast appends and pops on either end?,namedtuple(),namedtuple() factory function for creating tuple subclasses with named field...


In [116]:
# Clean the data

dataset['question'] = dataset['question'].apply(clean_text)
dataset['answer'] = dataset['answer'].apply(clean_text)
dataset['context'] = dataset['context'].apply(clean_text)

#### Selecting qns and answers with appropriate length (<20 words)

In [117]:
import pandas as pd

# Find the length of sentences
lengths = []
for question in clean_questions:
    lengths.append(len(question.split()))
for answer in clean_answers:
    lengths.append(len(answer.split()))
# Create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])

In [118]:
lengths['counts'].describe()

count    1378.000000
mean        7.045718
std         7.466513
min         0.000000
25%         2.000000
50%         6.000000
75%        10.000000
max       120.000000
Name: counts, dtype: float64

In [119]:
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))
print(np.percentile(lengths, 99))

11.0
12.0
13.0
16.0
34.0


In [131]:
dataset['question'].str.split()

0                        [what, module, implements, specialized, container, datatypes]
1                       [what, are, pythongs, general, purpose, built, in, containers]
2      [what, is, the, factory, function, for, creating, tuple, subclasses, with, n...
3      [what, is, a, deque, listlike, container, with, fast, appends, and, pops, on...
4                                     [chainmap, dictlike, class, for, creating, what]
                                            ...                                       
684       [what, is, accessible, via, the, data, attribute, of, userstring, instances]
685    [what, can, be, converted, into, a, string, using, the, builtin, str, function]
686                                         [what, do, userstring, instances, provide]
687    [what, is, a, real, str, object, used, to, store, the, contents, of, the, us...
688                                 [what, new, methods, were, added, in, version, 35]
Name: question, Length: 689, dtype: object

In [135]:
# Remove questions and answers that are shorter than 1 word and longer than 20 words.
min_line_length = 5
max_line_length = int(lengths['counts'].describe()['max'])

# Filter out the questions that are too short/long
short_questions_temp = []
short_answers_temp = []

short_dataset = dataset.loc[(dataset['question'].str.split().apply(len) > min_line_length) &
                            (dataset['question'].str.split().apply(len) < max_line_length), :]

len(short_dataset)

611

In [140]:
short_dataset.to_csv('collections_time_logging.csv')

# Part 2
### seq2seq word based (RNN)
Yes we know that RNNs have problem with long-term sequences, nevertheless we might find\
application for some parts like text summarization

#### Preprocess for word based model

In [141]:
#choosing number of samples
num_samples = len(short_answers)  # Number of samples to train on.
short_questions = short_questions[:num_samples]
short_answers = short_answers[:num_samples]

NameError: name 'short_answers' is not defined

In [142]:
#append start and end tokens for the answers
short_answers2 = []
for ans in short_answers:
    ans = '<SOS> ' + ans + ' <EOS>'
    short_answers2.append(ans)

NameError: name 'short_answers' is not defined

In [ ]:
# Create a dictionary for the frequency of the vocabulary
# Create
vocab = {}
for question in short_questions:
    for word in question.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

for answer in short_answers2:
    for word in answer.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

In [ ]:
# Remove rare words from the vocabulary.
# We aim to replace fewer than 5% of words with <UNK>

threshold = 2
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

In [ ]:
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

In [ ]:
#we will create dictionaries to provide a unique integer for each word.
vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        vocab_to_int[word] = word_num
        word_num += 1

In [ ]:
# Add the unique tokens (pad and unknown vocab) to the vocabulary dictionaries.
codes = ['<PAD>','<UNK>']
for code in codes:
    code_int = len(vocab_to_int)
    vocab_to_int[code] = code_int

In [ ]:
#switch <PAD> value to well's value of 0 for padding purposes later
print (vocab_to_int['<PAD>'])
for i, v in vocab_to_int.items():
    if v == 0:
        print (i)

In [ ]:
for i, v in vocab_to_int.items():
    if v == 0:
        vocab_to_int[i] = vocab_to_int['<PAD>']
vocab_to_int['<PAD>'] = 0

In [ ]:
# Create dictionaries to map the unique integers to their respective words.
# i.e. an inverse dictionary for vocab_to_int.
int_to_vocab = {v_i: v for v, v_i in vocab_to_int.items()}

In [ ]:
# Check the length of the dictionaries.
print(len(vocab_to_int))
print(len(int_to_vocab))

In [ ]:
# Convert the text to integers with paddings
# Replace any words that are not in the respective vocabulary with <UNK>
questions_int = []
for question in short_questions:
    ints = []
    for word in question.split():
        if word not in vocab_to_int:
            ints.append(vocab_to_int['<UNK>'])
        else:
            ints.append(vocab_to_int[word])
    questions_int.append(ints)

answers_int = []
for answer in short_answers2:
    ints = []
    for word in answer.split():
        if word not in vocab_to_int:
            ints.append(vocab_to_int['<UNK>'])
        else:
            ints.append(vocab_to_int[word])
    answers_int.append(ints)
# Check the lengths
print(len(questions_int))
print(len(answers_int))

In [ ]:
# Calculate what percentage of all words have been replaced with <UNK># Calcul
word_count = 0
unk_count = 0

for question in questions_int:
    for word in question:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1

for answer in answers_int:
    for word in answer:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1

unk_ratio = round(unk_count/word_count,4)*100

print("Total number of words:", word_count)
print("Number of times <UNK> is used:", unk_count)
print("Percent of words that are <UNK>: {}%".format(round(unk_ratio,3)))

In [ ]:
from keras_preprocessing.sequence import pad_sequences

#include padding
encoder_input_data = pad_sequences(questions_int, maxlen=max_line_length, value=vocab_to_int['<PAD>'], padding='post') #pad to max_line_length
decoder_input_data = pad_sequences(answers_int, maxlen=max_line_length+2, value=vocab_to_int['<PAD>'], padding='post') #pad to max_line_length + start and end tokens

In [ ]:
#decoder target is 1 timestep (word) ahead of decoder input, in a 3-d array
decoder_target_data = np.zeros(
    (len(answers_int), max_line_length+2, len(vocab_to_int)), #memory error occurs after 3500
    dtype='float32')

In [ ]:
for i, target_seq in enumerate(answers_int):
    for t, seq in enumerate(target_seq):
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, seq] = 1.

In [ ]:
print (encoder_input_data.shape)
print (decoder_input_data.shape)
print (decoder_target_data.shape)

In [ ]:
#include embedding size
embedding_size = 200

In [ ]:
from keras.layers import Input, Embedding, Bidirectional, LSTM, concatenate, Dense

In [ ]:
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(len(vocab_to_int), embedding_size)(encoder_inputs)
encoder = Bidirectional(LSTM(100, return_state=True))
encoder_outputs, state_h_1, state_c_1, state_h_2, state_c_2 = encoder(en_x)

In [ ]:
state_h = concatenate([state_h_1, state_h_2], axis=1)
state_c = concatenate([state_c_1, state_c_1], axis=1)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
#non-bidirectional approach
# encoder_inputs = Input(shape=(None,))
# en_x=  Embedding(len(vocab_to_int), embedding_size)(encoder_inputs)
# encoder = LSTM(50, return_state=True)
# encoder_outputs, state_h, state_c = encoder(en_x)
# # We discard `encoder_outputs` and only keep the states.
# encoder_states = [state_h, state_c]

In [ ]:
from keras.models import Model

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dex=  Embedding(len(vocab_to_int), embedding_size)
final_dex= dex(decoder_inputs)
decoder_lstm = LSTM(200, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)
decoder_dense = Dense(len(vocab_to_int), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc']) #sparse_categorical_crossentropy as labels in a single integer array

In [ ]:
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [ ]:
#early stopping & saving
# earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('Best_weights_collections.hdf5', save_best_only=True, monitor='val_loss', mode='min')
# reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')

In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=40,
          validation_split=0.05,
          callbacks= [mcp_save]
#           callbacks=[earlyStopping, mcp_save, reduce_lr_loss] #change from 0.1)))
         )

In [ ]:
model.save('s2s_collections_40.h5')

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

In [ ]:
#Create sampling model
decoder_state_input_h  = Input(shape=(200,))
decoder_state_input_c = Input(shape=(200,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = vocab_to_int['<SOS>']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = int_to_vocab[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '<EOS>' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for i in range(50):
    seq_index = np.random.randint(1, len(encoder_input_data))
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', short_answers[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)